<h1 align = "center"> Home Value Predictions in Greater Boston Area </h1> 

<img src="static/img/mls_listing.png" width=800 align="center">

<h6 align = "center"> MLS Listing Preview </h6> 

## Introduction

A home is often the largest and most expensive purchase a person makes in his or her lifetime. Ensuring homeowners have a trusted way to monitor this asset is incredibly important. While many individual homebuyers are less sensitive to price forecasts, small errors in price prediction can have systemic negative effects in the economy as a whole. Accurate prediction makes it easier to understand which features would influence the final property price.

## Problem Statement

Real estate prices are very much dependent on factors that are not easy to control. Analyzing broader market conditions and specific property determinants in order to establish how property values may change over the course of time are utterly important. Massive data can be obtained about the current market situation, which demands using powerful machine learning algorithms in order to predict with high precision and in a reasonable time frame.

*Goal: Explore different models using internal data source (MLS data). Choose the best model as the final home valuation algorithms to predict prices on a new set of listings. Use price predictions and other listing features to build the app for the "Midterm_Part_II".*

### Load libraries

In [1]:
import pandas as pd
import numpy as np

### Load Data

Before we start with modelling, we have to take a closer look at our data features:

In [2]:
# Read dataset columns
all_columns = pd.read_csv('static/data/sold_geocoded.csv').columns.values
all_columns

/Users/YangyangLi/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


array(['Unnamed: 0', 'DisplayX', 'DisplayY', 'ADDRESS', 'CITY', 'STATE',
       'ZIP', 'AGE', 'AGENTNAME', 'AREA', 'BATHS', 'BEDS', 'COOLING',
       'DOM', 'DTO', 'ELEMENTARYSCHOOL', 'EXPIREDDATE', 'GARAGE',
       'HEATING', 'HIGHSCHOOL', 'HOUSENUM1', 'HOUSENUM2',
       'JUNIORHIGHSCHOOL', 'LEVEL', 'LISTDATE', 'LISTPRICE', 'LOTSIZE',
       'MLSNUM', 'MONTH_ORIG', 'OFFICENAME', 'OFFICEPHONE',
       'OTHERFEATURES', 'PHOTOURL', 'PROPTYPE', 'REMARKS',
       'SHOWINGINSTRUCTIONS', 'SOLDDATE', 'SOLDPRICE', 'SQFT', 'STATUS',
       'STREETNAME', 'STYLE', 'TYPE', 'YEAR_ORIG', 'zoMLSNUM',
       'META_STATUS', 'MOS', 'Percent_diff', 'PPSF'], dtype=object)

Clearly, there are too many features in the dataset. We don't need all of them for this task. Let us choose a good set of predictors. 

In [3]:
use_cols = ['MLSNUM','DisplayX','DisplayY','ZIP','AGE','BATHS','BEDS','GARAGE','LISTDATE',
            'LISTPRICE','LOTSIZE','PROPTYPE','SOLDPRICE','SQFT', 'PPSF', 'PHOTOURL', 'REMARKS']

Data description:

    1. MLSNUM - listing number
    2. DisplayX - latitude
    3. DisplayY - longitude
    4. ZIP - ZIP code
    5. AGE - age of the building
    6. BATHS - number of bathrooms 
    7. BEDS - number of bedrooms 
    8. GARAGE - number of parking spots in the building
    9. LISTDATE - listed date of sale
    10. LISTPRICE - asking price
    11. LOTSIZE - building lotsize
    12. PROPTYPE - building type (CC-condo; MF-multifamily, SF-single family)
    13. SOLDPRICE - condo final price
    14. SQFT - total condo square feet
    15. PPSF - price per square foot
    16. PHOTOURL - single condo picture URL
    17. REMARKS - descriptions

In [4]:
# Load data with selected set of features
df_all = pd.read_csv('static/data/sold_geocoded.csv', usecols=use_cols)
df_all.head()

DisplayX   DisplayY   ZIP    AGE  BATHS  BEDS  GARAGE            LISTDATE  \
0 -71.003531  42.834870  1860   12.0    1.0     2     2.0   3/26/2013 0:00:00   
1 -71.893472  42.199845  1524    3.0    2.0     2     2.0   3/30/2013 0:00:00   
2 -71.437349  42.653706  1879    5.0    2.5     2     2.0  10/10/2013 0:00:00   
3 -70.888071  42.507990  1970  117.0    2.0     2     1.0  10/21/2013 0:00:00   
4 -70.904487  42.518712  1970   38.0    1.0     2     0.0   4/15/2014 0:00:00   

   LISTPRICE    LOTSIZE      MLSNUM  \
0   169900.0        NaN  71498924.0   
1   279900.0  1306800.0  71500500.0   
2   449900.0     5000.0  71595485.0   
3   389500.0     8400.0  71599223.0   
4   237000.0        NaN  71661544.0   

                                          PHOTOURL PROPTYPE  \
0  http://media.mlspin.com/photo.aspx?mls=71498924       CC   
1  http://media.mlspin.com/photo.aspx?mls=71500500       CC   
2  http://media.mlspin.com/photo.aspx?mls=71595485       CC   
3  http://media.mlspin.com/photo.aspx?mls=71599223       CC   
4  http://media.mlspin.com/photo.aspx?mls=71661544       CC   

                                             REMARKS  SOLDPRICE    SQFT  \
0  Grange Hall Condominiums in downtown Merrimac....   177500.0  1283.0   
1  2”X6” WALL CONSTRUCTION-ENERGY EFFICIENT HOMES...   295850.0  1368.0   
2  This 1.850 sq.ft. To be Built Pine Valley II s...   470519.0  1850.0   
3  Salem - Water view Condo in renovated quality ...   383500.0  1502.0   
4  Welcome to this two bedroom townhouse located ...   232000.0  1469.0   

         PPSF  
0  138.347623  
1  216.264620  
2  254.334595  
3  255.326232  
4  157.930565

### Data Preprocessing

This section includes preprocessing, technique necessary for the raw data transformation into an understandable format. MLS data may be often incomplete, inconsistent, and/or lacking in certain infomration, and is likely to contain many errors. Data preprocessing is a proven method of resolving such issues.

In [5]:
# Clean PPSF data
# PPSF has many "inf" values that need to be removed
# Take a look at the function below and 
# make sure there are no any "nan" or "inf" values left

df_all['PPSF'] = df_all['PPSF'].round(2)

# array to store indices
indexes = []

# colect all indices that contain "inf" value
for p in enumerate(df_all['PPSF']):
    if p[1] == np.inf:
        indexes.append(p[0])
        
# drop all rows with "inf" values        
df_all = df_all.drop(df_all.index[indexes])  

In [6]:
# Check the shape of the data - you should be able to see (158874, 17)
df_all.shape

(158874, 17)

YOUR TURN <br><br>
**Question 1 (1 pt):** <br> *Create new column 'LISTMONTH', extracting the information from the 'LISTDATE' feature.*

In [7]:
# YOUR CODE HERE

#converting
df_all['LISTDATE'] = pd.to_datetime(df_all['LISTDATE'])

#New column
df_all['LISTMONTH'] = df_all['LISTDATE'].apply(lambda x: x.month)



In [8]:
# Check datatypes
df_all.dtypes

DisplayX            float64
DisplayY            float64
ZIP                   int64
AGE                 float64
BATHS               float64
BEDS                  int64
GARAGE              float64
LISTDATE     datetime64[ns]
LISTPRICE           float64
LOTSIZE             float64
MLSNUM              float64
PHOTOURL             object
PROPTYPE             object
REMARKS              object
SOLDPRICE           float64
SQFT                float64
PPSF                float64
LISTMONTH             int64
dtype: object

The above output should be exactly the same as this one:<br>
<img src="static/img/dtypes.png" align="left">

YOUR TURN <br> <br> **Question 2 (1 pt):** <br> *One-hot encode 'ZIP' feature, using pandas get_dummies() method.*

In [9]:
df_all.head()

DisplayX   DisplayY   ZIP    AGE  BATHS  BEDS  GARAGE   LISTDATE  \
0 -71.003531  42.834870  1860   12.0    1.0     2     2.0 2013-03-26   
1 -71.893472  42.199845  1524    3.0    2.0     2     2.0 2013-03-30   
2 -71.437349  42.653706  1879    5.0    2.5     2     2.0 2013-10-10   
3 -70.888071  42.507990  1970  117.0    2.0     2     1.0 2013-10-21   
4 -70.904487  42.518712  1970   38.0    1.0     2     0.0 2014-04-15   

   LISTPRICE    LOTSIZE      MLSNUM  \
0   169900.0        NaN  71498924.0   
1   279900.0  1306800.0  71500500.0   
2   449900.0     5000.0  71595485.0   
3   389500.0     8400.0  71599223.0   
4   237000.0        NaN  71661544.0   

                                          PHOTOURL PROPTYPE  \
0  http://media.mlspin.com/photo.aspx?mls=71498924       CC   
1  http://media.mlspin.com/photo.aspx?mls=71500500       CC   
2  http://media.mlspin.com/photo.aspx?mls=71595485       CC   
3  http://media.mlspin.com/photo.aspx?mls=71599223       CC   
4  http://media.mlspin.com/photo.aspx?mls=71661544       CC   

                                             REMARKS  SOLDPRICE    SQFT  \
0  Grange Hall Condominiums in downtown Merrimac....   177500.0  1283.0   
1  2”X6” WALL CONSTRUCTION-ENERGY EFFICIENT HOMES...   295850.0  1368.0   
2  This 1.850 sq.ft. To be Built Pine Valley II s...   470519.0  1850.0   
3  Salem - Water view Condo in renovated quality ...   383500.0  1502.0   
4  Welcome to this two bedroom townhouse located ...   232000.0  1469.0   

     PPSF  LISTMONTH  
0  138.35          3  
1  216.26          3  
2  254.33         10  
3  255.33         10  
4  157.93          4

In [10]:

#There are negative numbers in age, this doesn't seem right.  
# SHOULD REMOVE?

#print("Age is less than zero= ", df_all[df_all.AGE < 0].shape)
#print ("removing less than zero values")

#df_all = df_all[df_all.AGE >= 0]

#print("Now age is less than zero= ", df_all[df_all.AGE < 0].shape)

In [11]:
# YOUR CODE HERE
#fix zip to make it five numbers

df_all['ZIP'] = df_all['ZIP'].apply(lambda code: '{:0>5s}'.format(str(code)))
print ("values less than five ints: ")
print (df_all.ZIP[df_all.ZIP.str.len() < 5])

#One hot encoding
df_all = pd.get_dummies(df_all, columns=['ZIP'])
print (df_all.shape)
df_all.head()


values less than five ints: 
Series([], Name: ZIP, dtype: object)
(158874, 725)


DisplayX   DisplayY    AGE  BATHS  BEDS  GARAGE   LISTDATE  LISTPRICE  \
0 -71.003531  42.834870   12.0    1.0     2     2.0 2013-03-26   169900.0   
1 -71.893472  42.199845    3.0    2.0     2     2.0 2013-03-30   279900.0   
2 -71.437349  42.653706    5.0    2.5     2     2.0 2013-10-10   449900.0   
3 -70.888071  42.507990  117.0    2.0     2     1.0 2013-10-21   389500.0   
4 -70.904487  42.518712   38.0    1.0     2     0.0 2014-04-15   237000.0   

     LOTSIZE      MLSNUM  ... ZIP_20184 ZIP_20302 ZIP_20382  ZIP_20452  \
0        NaN  71498924.0  ...         0         0         0          0   
1  1306800.0  71500500.0  ...         0         0         0          0   
2     5000.0  71595485.0  ...         0         0         0          0   
3     8400.0  71599223.0  ...         0         0         0          0   
4        NaN  71661544.0  ...         0         0         0          0   

   ZIP_20743  ZIP_20771  ZIP_20779  ZIP_92148  ZIP_92664  ZIP_92962  
0          0          0          0          0          0          0  
1          0          0          0          0          0          0  
2          0          0          0          0          0          0  
3          0          0          0          0          0          0  
4          0          0          0          0          0          0  

[5 rows x 725 columns]

In [12]:
# The final cleaning step is to take care of missing values
# Drop all "nan" rows/cols using .dropna() pandas method
df_all.dropna(inplace=True)

In [13]:
# Final check for "nan" values
# Notice the use of .head(18) here!!!
# We are preventing long outputs and
# try to print only columns that corespond to "use_cols" features set
# Important: Remove .head(18) to check all features

print (df_all.isnull().sum())
#print (df_all.isnull().sum() > 1)
#print ((df_all.isna().sum() > 0) == TRUE)
#df_all.loc[:, df_all.isna().any()]


DisplayX     0
DisplayY     0
AGE          0
BATHS        0
BEDS         0
GARAGE       0
LISTDATE     0
LISTPRICE    0
LOTSIZE      0
MLSNUM       0
PHOTOURL     0
PROPTYPE     0
REMARKS      0
SOLDPRICE    0
SQFT         0
PPSF         0
LISTMONTH    0
ZIP_00000    0
ZIP_00161    0
ZIP_00180    0
ZIP_00185    0
ZIP_00198    0
ZIP_00211    0
ZIP_00216    0
ZIP_00218    0
ZIP_00256    0
ZIP_00263    0
ZIP_01000    0
ZIP_01001    0
ZIP_01002    0
            ..
ZIP_03034    0
ZIP_03858    0
ZIP_04120    0
ZIP_05510    0
ZIP_07766    0
ZIP_08145    0
ZIP_08164    0
ZIP_08835    0
ZIP_08861    0
ZIP_09107    0
ZIP_10002    0
ZIP_10018    0
ZIP_10157    0
ZIP_10175    0
ZIP_10468    0
ZIP_10603    0
ZIP_10826    0
ZIP_10902    0
ZIP_10915    0
ZIP_20155    0
ZIP_20184    0
ZIP_20302    0
ZIP_20382    0
ZIP_20452    0
ZIP_20743    0
ZIP_20771    0
ZIP_20779    0
ZIP_92148    0
ZIP_92664    0
ZIP_92962    0
Length: 725, dtype: int64


YOUR TURN <br><br>
**Question 3 (1 pt):** <br> *Extract condos data and create the new dataframe called "condos".* 
<br><br>Hint: PROPTYPE has 3 unique values: 'CC','MF', and 'CC', where 'CC' stands for **condos**.

In [14]:
# YOUR CODE HERE
condos = df_all[df_all.PROPTYPE == 'CC']
print(condos.PROPTYPE.shape)
print(df_all.PROPTYPE.shape)

(10027,)
(127185,)


Let us sort all condo values by 'LISTDATE' and check the final shape of the data:

In [15]:
# Sort all values by 'LISTDATE'
condos = condos.sort_values(by='LISTDATE')
condos.head()

DisplayX   DisplayY    AGE  BATHS  BEDS  GARAGE   LISTDATE  \
122119 -70.935221  42.497680   31.0    2.5     3     1.0 2012-05-06   
23494  -71.435071  42.652326    2.0    2.5     2     2.0 2013-02-23   
1      -71.893472  42.199845    3.0    2.0     2     2.0 2013-03-30   
113725 -71.316857  42.362081    3.0    3.5     3     2.0 2013-04-28   
13698  -70.934101  41.543220  118.0    2.5     2     0.0 2013-06-21   

        LISTPRICE    LOTSIZE      MLSNUM  ... ZIP_20184 ZIP_20302 ZIP_20382  \
122119   295000.0   938718.0  71378749.0  ...         0         0         0   
23494    612799.0     4597.0  72109900.0  ...         0         0         0   
1        279900.0  1306800.0  71500500.0  ...         0         0         0   
113725  2795000.0        0.0  71515996.0  ...         0         0         0   
13698    379999.0        0.0  71545648.0  ...         0         0         0   

        ZIP_20452  ZIP_20743  ZIP_20771  ZIP_20779  ZIP_92148  ZIP_92664  \
122119          0          0          0          0          0          0   
23494           0          0          0          0          0          0   
1               0          0          0          0          0          0   
113725          0          0          0          0          0          0   
13698           0          0          0          0          0          0   

        ZIP_92962  
122119          0  
23494           0  
1               0  
113725          0  
13698           0  

[5 rows x 725 columns]

In [16]:
# Condos shape - you shoud see (10027, 725)
condos.shape

(10027, 725)

### Split data into "future_data" and "training_data"

So far we have worked on cleaning the data. Now, we want to split it into two datasets: "training_data" and "future_data". The first dataset ("training_data") as its name implies will serve for training and second dataset  ("future_data") will act like a future, "unseen" data. The final (best) model will be used to predict prices on "future_data" dataset.

In [17]:
# Create "future_data" dataset
# Future data does not have SOLDPRICE and thus, the feature should be excluded
# NOTE: Future data will have 60 listings - see how we slice the data 
# Export data as "midterm_future_data.csv", storing the dataset in your "data" folder

future_data = condos[9967:]
future_data_features = list(set(future_data.columns) - set(['SOLDPRICE'])) 
future_data = future_data[future_data_features]
future_data.to_csv('static/data/midterm_future_data.csv', index=None)

In [18]:
# Check future_data shape - you shoud see (60, 724)
future_data.shape
#condos.dtypes

(60, 724)

YOUR TURN <br><br>
**Question 4 (1 pt):** <br> *Create "training_data.csv" dataset and store it into your "data" folder.* <br><br>Training data should contain all data except the last 60 obsevrations we have periously exported as "midterm_future_data.csv" data. 

In [19]:
# YOUR CODE HERE
training_data = condos[:9967]
training_data.to_csv('static/data/midterm_training_data.csv', index=None)




In [20]:
# Check the shape of your data - you shoud see (9967, 725)
training_data.shape

(9967, 725)

Before we move on, make sure you have "midterm_future_data.csv" and "midterm_training_data.csv" datasets stored into your "data" folder (they will be together with you "sold_geocoded.csv" dataset):

<img src="static/img/datasets.png" align="left">

## Data Modeling

In this section we are going to train, test, and choose the best performing model. The steps are the following:
    1. Load "midterm_training_data.csv" 
    2. Choose an appropriate feature set
    3. Train/Test data by using 2 models:
        - Random Forest
        - Gradient Boosting
    4. Choose better model and export predictions

#### Load "midterm_training_data.csv" dataset

In [21]:
# Load data
train = pd.read_csv('static/data/midterm_training_data.csv')
train.head()

DisplayX   DisplayY    AGE  BATHS  BEDS  GARAGE    LISTDATE  LISTPRICE  \
0 -70.935221  42.497680   31.0    2.5     3     1.0  2012-05-06   295000.0   
1 -71.435071  42.652326    2.0    2.5     2     2.0  2013-02-23   612799.0   
2 -71.893472  42.199845    3.0    2.0     2     2.0  2013-03-30   279900.0   
3 -71.316857  42.362081    3.0    3.5     3     2.0  2013-04-28  2795000.0   
4 -70.934101  41.543220  118.0    2.5     2     0.0  2013-06-21   379999.0   

     LOTSIZE      MLSNUM  ... ZIP_20184 ZIP_20302 ZIP_20382  ZIP_20452  \
0   938718.0  71378749.0  ...         0         0         0          0   
1     4597.0  72109900.0  ...         0         0         0          0   
2  1306800.0  71500500.0  ...         0         0         0          0   
3        0.0  71515996.0  ...         0         0         0          0   
4        0.0  71545648.0  ...         0         0         0          0   

   ZIP_20743  ZIP_20771  ZIP_20779  ZIP_92148  ZIP_92664  ZIP_92962  
0          0          0          0          0          0          0  
1          0          0          0          0          0          0  
2          0          0          0          0          0          0  
3          0          0          0          0          0          0  
4          0          0          0          0          0          0  

[5 rows x 725 columns]

#### Select features

Exclude 'LISTPRICE','SOLDPRICE', and 'LISTDATE' given that, these are unknown features for any given listing in the future. 'DisplayX'and 'DisplayY' cannot be used as predictors. Usually, they have been used to create distance matrices necessary for estimating the nearest neighbors, which is outside of the scope of this work. 'PHOTOURL' are URL links and 'MLSNUM' is a Database ID. 'REMARKS' are descritpions (we are not doing text analysis here) and thus we are exclusing them as well. We have already extracted condos ('CC') from the 'PROPTYPE' feature.

In [22]:
# Create train_features valiable and store selected features in it
train_features = list(set(train.columns) - set(['PHOTOURL','MLSNUM','SOLDPRICE','REMARKS','LISTDATE',
                                          'LISTPRICE','PROPTYPE', 'DisplayX','DisplayY']))

#### Train/Test data

In [23]:
# Load sklearn library
from sklearn.model_selection import train_test_split

YOUR TURN <br><br>
**Question 5 (1 pt):** <br> *Train - Test - **Split** * <br><br>Use sklearn to split the train dataset into train and test. Specifically, create 'X_train', 'X_test', 'y_train', and 'y_test' datasets. Use 70% for the train and 30% for the test data. Come up with your unique random_state number."  

In [24]:
# YOUR CODE
response_col = 'SOLDPRICE'
test_size = 0.3
random_state = 7577

X_train, X_test, y_train, y_test = train_test_split(train[train_features], train[response_col], test_size=test_size, random_state=random_state)

In [25]:
# Check the shape of the data
# You should get: 
# X_train.shape = (6976, 716), y_train.shape = (6976,)
# X_test.shape = (2991, 716), y_test.shape = (2991,)

print('X_train.shape = {}, y_train.shape = {}'.format(X_train.shape, y_train.shape))
print('X_test.shape = {}, y_test.shape = {}'.format(X_test.shape, y_test.shape))

X_train.shape = (6976, 716), y_train.shape = (6976,)
X_test.shape = (2991, 716), y_test.shape = (2991,)


### Training Your Models

In [26]:
# Load library
from sklearn.ensemble import RandomForestRegressor

YOUR TURN <br><br>
**Question 6 (1 pt):** <br> *Train data using Random Forest model.*

In [27]:
# YOUR CODE HERE
rf = RandomForestRegressor(random_state=10)
rf.fit(X_train, y_train)

print('R2 score on train: ', rf.score(X_train, y_train))
print('R2 score on test: ', rf.score(X_test, y_test))



/Users/YangyangLi/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


R2 score on train:  0.9968308341631875
R2 score on test:  0.9657911006251607


In [28]:
# Load library
from sklearn.ensemble import GradientBoostingRegressor

YOUR TURN <br><br>
**Question 7 (1 pt):** <br> *Train data using Gradient Boosting model.*

In [29]:
# YOUR CODE HERE
gb = GradientBoostingRegressor().fit(X_train, y_train)
gb.score(X_test, y_test)


0.9842047464553605

### Predict on Future Data

In [30]:
# Import "future_data" dataset
future_data = pd.read_csv("static/data/midterm_future_data.csv")
future_data.head()  

ZIP_01701  ZIP_01073  ZIP_01529  ZIP_01824  ZIP_01906  ZIP_02117  \
0          0          0          0          0          0          0   
1          0          0          0          0          0          0   
2          0          0          0          0          0          0   
3          0          0          0          0          0          0   
4          0          0          0          0          0          0   

   ZIP_01831  ZIP_01368  ZIP_02464  ZIP_02347  ...  ZIP_02574  ZIP_02051  \
0          0          0          0          0  ...          0          0   
1          0          0          0          0  ...          0          0   
2          0          0          0          0  ...          0          0   
3          0          0          0          0  ...          0          0   
4          0          0          0          0  ...          0          0   

   ZIP_02351  ZIP_02342  ZIP_01009  ZIP_20382  ZIP_02068  ZIP_01049  \
0          0          0          0          0          0          0   
1          0          0          0          0          0          0   
2          0          0          0          0          0          0   
3          0          0          0          0          0          0   
4          0          0          0          0          0          0   

   ZIP_02446  ZIP_02370  
0          0          0  
1          0          0  
2          0          0  
3          0          0  
4          0          0  

[5 rows x 724 columns]

In [31]:
# Print future_data columns 
# You can use future_data.columns.values 

future_data.columns

Index(['ZIP_01701', 'ZIP_01073', 'ZIP_01529', 'ZIP_01824', 'ZIP_01906',
       'ZIP_02117', 'ZIP_01831', 'ZIP_01368', 'ZIP_02464', 'ZIP_02347',
       ...
       'ZIP_02574', 'ZIP_02051', 'ZIP_02351', 'ZIP_02342', 'ZIP_01009',
       'ZIP_20382', 'ZIP_02068', 'ZIP_01049', 'ZIP_02446', 'ZIP_02370'],
      dtype='object', length=724)

We want to extract only valid predictors from "future_data" dataset. Note: we don't want to use features such as 'PHOTOURL','MLSNUM' or 'LISTPRICE' as our predictors.

YOUR TURN <br><br>
**Question 8 (1 pt):** <br> Create "predicion_features" variable to store the approprate set of features. <br>Note: The number of features needs to coresspond to the features set used for training. 

In [32]:
# YOUR CODE HERE
prediction_features = list(set(future_data.columns) - set(['PHOTOURL','MLSNUM','SOLDPRICE','REMARKS','LISTDATE',
                                          'LISTPRICE','PROPTYPE', 'DisplayX','DisplayY']))


prediction_features

['ZIP_01701',
 'ZIP_01824',
 'ZIP_01906',
 'ZIP_01529',
 'ZIP_01073',
 'ZIP_02117',
 'ZIP_01831',
 'ZIP_01368',
 'ZIP_02464',
 'ZIP_02669',
 'ZIP_02347',
 'ZIP_01084',
 'ZIP_01613',
 'ZIP_02332',
 'ZIP_01885',
 'ZIP_02118',
 'ZIP_01220',
 'ZIP_01522',
 'ZIP_01821',
 'ZIP_02858',
 'ZIP_02338',
 'ZIP_02452',
 'ZIP_01072',
 'ZIP_01566',
 'ZIP_01035',
 'ZIP_02476',
 'ZIP_02764',
 'ZIP_01245',
 'ZIP_01367',
 'ZIP_01374',
 'ZIP_92148',
 'ZIP_02547',
 'ZIP_02558',
 'ZIP_01037',
 'ZIP_01235',
 'ZIP_01515',
 'ZIP_01756',
 'ZIP_01564',
 'ZIP_01340',
 'AGE',
 'ZIP_02746',
 'ZIP_02745',
 'ZIP_01915',
 'ZIP_02402',
 'ZIP_01074',
 'ZIP_02144',
 'ZIP_02644',
 'ZIP_01378',
 'ZIP_01890',
 'ZIP_02676',
 'ZIP_01351',
 'ZIP_02062',
 'ZIP_02109',
 'ZIP_01508',
 'ZIP_01516',
 'ZIP_02115',
 'ZIP_01109',
 'ZIP_02673',
 'ZIP_02143',
 'ZIP_01931',
 'ZIP_01607',
 'ZIP_00198',
 'ZIP_02482',
 'ZIP_02651',
 'ZIP_02543',
 'ZIP_01521',
 'ZIP_01011',
 'GARAGE',
 'ZIP_01092',
 'ZIP_02767',
 'ZIP_02662',
 'ZIP_01961',
 

In [33]:
# Create new dataset called "predictions_df"
# The dataset will contain only "predicion_features" set

predictions_df = future_data[prediction_features]
#predictions_df.head()

predictions_df2 = predictions_df.iloc[0:5]
predictions_df2


ZIP_01701  ZIP_01824  ZIP_01906  ZIP_01529  ZIP_01073  ZIP_02117  \
0          0          0          0          0          0          0   
1          0          0          0          0          0          0   
2          0          0          0          0          0          0   
3          0          0          0          0          0          0   
4          0          0          0          0          0          0   

   ZIP_01831  ZIP_01368  ZIP_02464  ZIP_02669  ...  ZIP_02574  ZIP_02051  \
0          0          0          0          0  ...          0          0   
1          0          0          0          0  ...          0          0   
2          0          0          0          0  ...          0          0   
3          0          0          0          0  ...          0          0   
4          0          0          0          0  ...          0          0   

   ZIP_02351  ZIP_02342  ZIP_01009  ZIP_20382  ZIP_02068  ZIP_02135  \
0          0          0          0          0          0          0   
1          0          0          0          0          0          0   
2          0          0          0          0          0          0   
3          0          0          0          0          0          0   
4          0          0          0          0          0          0   

   ZIP_02446  ZIP_02370  
0          0          0  
1          0          0  
2          0          0  
3          0          0  
4          0          0  

[5 rows x 716 columns]

YOUR TURN <br><br>
**Question 9 (1 pt):** <br> Predict prices using your best model. Store predictions in the variable called "predictions". 

In [34]:
# YOUR CODE HERE
predictions = gb.predict(predictions_df)


predictions2 = gb.predict(predictions_df2)

#sing_pred = gb.predict([[4, 2.5, 3005, 15, 17903.0]])


#sing_pred = gb.predict([[4, 2.5, 3005, 15, 17903.0]])



In [35]:
# Round prediction values 
predictions = predictions.round(2)

predictions2 = predictions2.round(2)
predictions2

array([-17180.22, 112654.01, 173881.09,  96917.9 , 107507.07])

In [36]:
# Add new column 'predicted_price' to the 'future_data' dataset
# Pass newly predicted values

future_data['predicted_price'] = predictions

In [37]:
# Choose the final set of columns to be use for you final export

final_cols = ['DisplayY','DisplayX','BEDS','SQFT','PHOTOURL','PPSF',
            'BATHS','MLSNUM','LISTPRICE', 'REMARKS', 'predicted_price']

In [38]:
# Create new dataframe for export
# NOTE: This dataframe will be exported to csv file and used in Midterm_Part_II

for_export_df = future_data[final_cols]

In [39]:
# Change column names (small caps)
for_export_df = for_export_df.rename(columns={'REMARKS':'remarks', 'LISTPRICE':'list_price', 'DisplayY':'display_y', 
                                        'DisplayX':'display_x', 'BEDS':'beds', 'SQFT':'sqft', 'PHOTOURL':'photo_url', 
                                        'PPSF':'ppsf', 'BATHS':'baths', 'MLSNUM':'mlsnum'})

In [40]:
# Round lon/lat data
# Parse MLS number to int 
# Note: For MLS number, you may want to use string later, in the Midterm_Part_II 

for_export_df['display_y'] = for_export_df['display_y'].round(5)
for_export_df['display_x'] = for_export_df['display_x'].round(5)
for_export_df['mlsnum'] = for_export_df['mlsnum'].astype(int)

In [41]:
# Final check
for_export_df.head()

display_y  display_x  beds    sqft  \
0   42.34879  -71.07807     1   418.0   
1   42.36682  -71.03763     3  1426.0   
2   42.26208  -71.52786     2  2100.0   
3   42.35796  -71.05054     1  1230.0   
4   42.61405  -71.22847     2  1040.0   

                                         photo_url     ppsf  baths    mlsnum  \
0  http://media.mlspin.com/photo.aspx?mls=72250173  1244.02    1.0  72250173   
1  http://media.mlspin.com/photo.aspx?mls=72251181   490.18    2.0  72251181   
2  http://media.mlspin.com/photo.aspx?mls=72250996   189.05    2.5  72250996   
3  http://media.mlspin.com/photo.aspx?mls=72254711  1341.46    1.5  72254711   
4  http://media.mlspin.com/photo.aspx?mls=72250876   302.88    2.5  72250876   

   list_price                                            remarks  \
0    519000.0  Excellent opportunity to own a studio at Trini...   
1    699000.0  Newly renovated, 3 bedroom +study, 2 bath feat...   
2    409000.0  Lovely townhome in The Preserve a gated commun...   
3   1650000.0  Seldom available opportunity to have the covet...   
4    304900.0  Looking for that well priced Townhouse that is...   

   predicted_price  
0        -17180.22  
1        112654.01  
2        173881.09  
3         96917.90  
4        107507.07

Your dataset (from above) should look like this (**'predicted_price' figures may be different due to dfferent splits and that is ok! **):<br>
<img src="static/img/final_dataset.png" align="center">

YOUR TURN <br><br>
**Question 10 (1 pt):** <br> Export "for_export_df" dataframe into "midterm.csv" file and store it into your "data" folder. You will model the "condos" database table according to it, so make sure you have completed this step!

In [42]:
# YOUR CODE HERE
for_export_df.to_csv('static/data/midterm.csv', index=None)



In [43]:
# Check the dataset shape - you should see (60, 11)
for_export_df.shape
print("ready player 1")


ready player 1


Your "data" folder should look like this (together with "sold_geocoded.csv"):

<img src="static/img/data2.png" align="left">

Congratulations! You finished the Part I. :)

In [44]:
for_export_df.dtypes

display_y          float64
display_x          float64
beds                 int64
sqft               float64
photo_url           object
ppsf               float64
baths              float64
mlsnum               int64
list_price         float64
remarks             object
predicted_price    float64
dtype: object

In [45]:
# Export model

from sklearn.externals import joblib
joblib.dump(gb, 'gb.pkl')

['gb.pkl']